In [1]:
from random import randint
import numpy as np
from PIL import Image, ImageDraw
import os
images = []
imagecodes = []
classnames = ['axes','boots','carabiners']
def listdir(path, list_name,filename):  #传入存储的list
    for file in os.listdir(path):  
        file_path = os.path.join(path, file)  
        if os.path.isdir(file_path):  
            listdir(file_path, list_name,file)  
        else:
            images.append(np.array(Image.open(file_path)))
            if filename =='axes':
                imagecodes.append(0)
            elif filename =='boots':
                imagecodes.append(1)
            elif filename =='carabiners':
                imagecodes.append(2)
            list_name.append(file_path)
    return images, imagecodes
images, imagecodes = listdir("images/train",[],"")
print(images)
print(imagecodes)

[array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]

In [2]:
# Set the Keras backend to CNTK
from keras import backend as K
import os
from importlib import reload

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("tensorflow")


f:\python\python3.6\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def preprocess_images(image_array):
    from PIL import Image, ImageOps
    
    # Pre-process all of the images to make them consistent
    images = []
    for img in image_array:
        # Equalize the pixel intensity to ensure consistent contrast
        img = ImageOps.equalize(Image.fromarray(img))
        # flatten the images
        images.append(np.array(img))
        
        # (our images are all the same size - otherwise you should resize them!)
        
    return images


from keras.utils import np_utils
from keras.utils import to_categorical
import numpy as np

# The images are our features, the imagecodes are the labels
features = np.array(preprocess_images(images))
labels = np.array(imagecodes)

#Format features
features = features.astype('float32')
features /= 255

# Format labels
labels = to_categorical(labels, len(classnames))
labels = labels.astype('float32')

# Show the shape of the features array (num_images, width, height, channels)
print (features.shape)

(369, 128, 128, 3)


In [4]:
# Train a CNN classifier
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

# Define the model as a sequence of layers
model = Sequential()

# The input layer accepts an image and applies a convolution that uses 32 6x6 filters and a rectified linear unit activation function
model.add(Conv2D(32, (6, 6), input_shape=(features.shape[1], features.shape[2], features.shape[3]), activation='relu'))

# Next we;ll add a max pooling layer with a 2x2 patch
model.add(MaxPooling2D(pool_size=(2,2)))

# A dropout layer randomly drops some nodes to reduce inter-dependencies (which can cause over-fitting)
model.add(Dropout(0.2))

# We can add as many layers as we think necessary - here we'll add another convolution, max pooling, and dropout layer
model.add(Conv2D(32, (6, 6), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# And another set
model.add(Conv2D(32, (6, 6), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Now we'll flatten the feature maps and generate an output layer with a predicted probability for each class
model.add(Flatten())
model.add(Dense(len(classnames), activation='softmax'))

# With the layers defined, we can ow compile the model for categorical (multi-class) classification
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [5]:
# Train the model over 3 epochs using 50-image batches and holding back the last 30% of the data for validation
model.fit(features, labels, epochs=3, batch_size=50, validation_split=0.3)

# Note: The validation_split parameter holds back the last X% (in this case 30%) of the training data WITHOUT shuffling the data.
# Our data is ordered such that instances the three classes are distributed evenly, so this is fine.
# If the data is ordered such that the the last X% doesn't contain a mix of all classes, you should either:
#   - Shuffle the data before training (ensuring features still match corresponding labels.)
#   - Randomly split the data first and use the validation_data parameter instead of validation_split

Train on 258 samples, validate on 111 samples
Epoch 1/3
258/258 [==============================] - 16s 62ms/step - loss: 2.7670 - acc: 0.3760 - val_loss: 0.9763 - val_acc: 1.0000
Epoch 2/3
258/258 [==============================] - 14s 56ms/step - loss: 1.0806 - acc: 0.4109 - val_loss: 0.4056 - val_acc: 1.0000
Epoch 3/3
258/258 [==============================] - 15s 57ms/step - loss: 1.1433 - acc: 0.5930 - val_loss: 1.0489 - val_acc: 0.6216


In [6]:
from random import randint
from PIL import Image, ImageOps


# Modify the image data to match the format of the training features
img = np.array(ImageOps.equalize(Image.open('data/21HnHt+LMDL._AC_US436_QL65_ (2).jpg')))
imgfeatures = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
imgfeatures = imgfeatures.astype('float32')
imgfeatures /= 255

# Use the classifier to predict the class
predicted_class = model.predict(imgfeatures)
# Print the predicted probabilities for each class
print(predicted_class)
# Find the class with the highest predicted probability
i = np.where(predicted_class == predicted_class.max())
print (classnames[int(i[1])])

[[0.46076432 0.31757993 0.2216557 ]]
axes
